# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 3-1. Задача TSP: инкрементальные алгоритмы.

В этой задаче Вам предлагается сравнить алгоритмы Nearest Neighbour и Nearest Insertion в задаче Euclidean TSP.

**Даны:**
* Координаты точек плоскости, являющихся вершинами графа.

**Найти:**
* Перестановку вершин, задающих минимальный по длине гамильтонов цикл в графе.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP.
* Реализуйте функции `solve_tsp_nearest_neighbour` и `solve_tsp_nearest_insertion`.
* Запустите функцию `run_all()`, чтобы протестировать свой код и сравнить качество решений, получаемых Nearest Neighbour и Nearest Insertion. Сильно ли они отличаются? Запишите свои качественные выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [1]:
def read_tsp_instance(filename):
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0,0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates

from math import sqrt

def euclidean_distance(point1, point2):
    return sqrt((point1[0]-point2[0]) ** 2 + (point1[1]-point2[1]) ** 2)
    
def calculate_tour_length(instance, permutation):
    n = len(permutation)
    return sum(euclidean_distance(instance[permutation[i]], instance[permutation[(i+1) % n]]) for i in range(len(permutation)))

In [2]:
import numpy as np

In [33]:
def solve_tsp_nearest_neighbour(instance):
    n = len(instance)
    candidates = {i for i in range(1, n)}
    permutation = [0]
    n = len(instance)
    for i in range(n - 1):
        tmp = list(candidates)
        next_v = np.argmin([euclidean_distance(instance[permutation[i]], instance[j]) for j in tmp])
        permutation.append(tmp[next_v])
        candidates.remove(tmp[next_v])
    return permutation

In [36]:
def solve_tsp_nearest_insertion(instance):
    n = len(instance)
    start = min([(euclidean_distance(instance[i], instance[j]), i, j) for i in range(n) for j in range(n) if i != j])
    permutation = [start[1], start[2]]
    candidates = {i for i in range(n) if i != permutation[0] and i != permutation[1]}
    for i in range(n - 2):
        tmp = list(candidates)
        dist = []
        for j in tmp:
            dist.append(np.min([euclidean_distance(instance[t], instance[j]) for t in permutation]))
        new_v = tmp[np.argmin(dist)]
        cur_len = len(permutation)
        edge = np.argmin([euclidean_distance(instance[new_v], instance[permutation[j]]) + 
                          euclidean_distance(instance[new_v], instance[permutation[(j + 1) % cur_len ]]) -
                          euclidean_distance(instance[permutation[j]], instance[permutation[(j + 1) % cur_len]])
                          for j in range(cur_len)])
        permutation.insert(edge + 1, new_v)
        candidates.remove(new_v)
    return permutation

In [31]:
import time
from os.path import exists

def run_all():
    instance_filenames = ['d198.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp', 'pr107.tsp', 'pr152.tsp', 'pr439.tsp']
    for filename in instance_filenames:
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality_nn = calculate_tour_length(instance, solve_tsp_nearest_neighbour(instance))
        time_nn = time.monotonic()-time_start
        time_start = time.monotonic()
        quality_ni = calculate_tour_length(instance, solve_tsp_nearest_insertion(instance))
        time_ni = time.monotonic()-time_start
        print(' done in {:.2} seconds with tour length {} using NN and in {:.2} seconds with tour length {} using NI'.format(time_nn, int(quality_nn), time_ni, int(quality_ni)))

In [37]:
run_all()

Solving instance d198.tsp… done in 0.013 seconds with tour length 18620 using NN and in 0.91 seconds with tour length 18050 using NI
Solving instance d493.tsp… done in 0.07 seconds with tour length 43646 using NN and in 1.2e+01 seconds with tour length 42130 using NI
Solving instance d657.tsp… done in 0.13 seconds with tour length 62176 using NN and in 2.7e+01 seconds with tour length 60081 using NI
Solving instance d2103.tsp… done in 1.2 seconds with tour length 87468 using NN and in 8.7e+02 seconds with tour length 85771 using NI
Solving instance pr107.tsp… done in 0.0042 seconds with tour length 46678 using NN and in 0.19 seconds with tour length 53211 using NI
Solving instance pr152.tsp… done in 0.0076 seconds with tour length 85702 using NN and in 0.48 seconds with tour length 88661 using NI
Solving instance pr439.tsp… done in 0.061 seconds with tour length 131282 using NN and in 1.2e+01 seconds with tour length 133705 using NI


## Выводы
Метод ближайшей вставки не всегда дает результат лучше, чем метод ближайшего соседа. А в тех случаях когда он лучше проигрышь ближайшего соседа составляет не больше 3,5 процентов, при чем время работы NI на порядок больше.